In [1]:
!pip install transformers
!pip install datasets

import os
import random
from sklearn.metrics import accuracy_score
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

import pandas as pd
from tqdm import tqdm

from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification

# for graphing
import seaborn as sns
import matplotlib.pyplot as plt

2023-02-16 12:20:57.740491: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
!pip install optimum[neural-compressor]
!pip install optimum[openvino,nncf]

zsh:1: no matches found: optimum[neural-compressor]
zsh:1: no matches found: optimum[openvino,nncf]


In [3]:
from datasets import load_dataset

dataset = load_dataset("klue", "ynat")

Found cached dataset klue (/Users/hwangtaegyeong/.cache/huggingface/datasets/klue/ynat/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# argment setting
model_checkpoint = "./FP32" #"bert-base-multilingual-cased"
batch_size_per_device = 32
max_length = 64
n_epochs = 3
warmup_ratio = .2
lr = 5e-5

In [5]:
# load model&tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

In [6]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')
# torch.device("cpu") 

No GPU available, using the CPU instead.


In [7]:
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

/var/folders/zb/vk00ck914_9fg7vznddlftwc0000gn/T/ipykernel_68813/2538770256.py:2: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


accuracy, bertscore, bleu, bleurt, brier_score, cer, character, charcut_mt, chrf, code_eval, comet, competition_math, coval, cuad, exact_match, f1, frugalscore, glue, google_bleu, indic_glue, mae, mahalanobis, mape, mase, matthews_correlation, mauve, mean_iou, meteor, mse, nist_mt, pearsonr, perplexity, poseval, precision, recall, rl_reliability, roc_auc, rouge, sacrebleu, sari, seqeval, smape, spearmanr, squad, squad_v2, super_glue, ter, trec_eval, wer, wiki_split, xnli, xtreme_s, BucketHeadP65/confusion_matrix, BucketHeadP65/roc_curve, Drunper/metrica_tesi, Felipehonorato/my_metric, GMFTBY/dailydialog_evaluate, GMFTBY/dailydialogevaluate, JP-SystemsX/nDCG, Josh98/nl2bash_m, KevinSpaghetti/accuracyk, NCSOFT/harim_plus, NikitaMartynov/spell-check-metric, NimaBoscarino/weat, Ochiroo/rouge_mn, Vertaix/vendiscore, Viona/infolm, Vlasta/pr_auc, abdusah/aradiawer, abidlabs/mean_iou, abidlabs/mean_iou2, angelina-wang/directional_bias_amplification, anz2/iliauniiccocrevaluation, bstrai/classif

In [8]:
metric_macrof1 = load_metric('f1')

/var/folders/zb/vk00ck914_9fg7vznddlftwc0000gn/T/ipykernel_68813/1490004349.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_macrof1 = load_metric('f1')


In [9]:
def compute_metrics(eval_pred):
    predictions = eval_pred.predictions.argmax(-1)
    labels = eval_pred.label_ids
    return metric_macrof1.compute(predictions=predictions,
                                  references=labels, average='macro')

In [10]:
!pip install evaluate

In [11]:
import evaluate
import numpy as np
from optimum.intel.openvino import OVConfig, OVModelForSequenceClassification, OVTrainer

model = BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=7)    
tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
dataset = load_dataset("klue", "ynat")

dataset = dataset.map(
    lambda examples: tokenizer(examples["title"], padding=True, truncation=True, max_length=max_length), batched=True
)

# The directory where the quantized model will be saved
save_dir = "qat"

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/nncf/common/utils/logger/__init__.py:17: FutureWarning: Importing from nncf.common.utils.logger is deprecated. Import `from nncf` directly instead, i.e.: 
`from nncf import set_log_level` instead of `from nncf.common.utils.logger import set_log_level`, and:
`from nncf import nncf_logger` instead of `from nncf.common.utils.logger import logger as nncf_logger`
  warning_deprecated(
Found cached dataset klue (/Users/hwangtaegyeong/.cache/huggingface/datasets/klue/ynat/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/hwangtaegyeong/.cache/huggingface/datasets/klue/ynat/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e/cache-4be0a577446603b6.arrow


  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
from transformers import TrainingArguments
from transformers import default_data_collator
# Load the default quantization configuration detailing the quantization we wish to apply
ov_config = OVConfig()

trainer = OVTrainer(
    model=model,
    args=TrainingArguments(save_dir, num_train_epochs=3.0, do_train=True, do_eval=True),
    train_dataset=dataset["train"].select(range(1000)),
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    ov_config=ov_config,
    task="sequence-classification",
)
train_result = trainer.train()
metrics = trainer.evaluate()
trainer.save_model()

optimized_model = OVModelForSequenceClassification.from_pretrained(save_dir)

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: date, guid, url, title. If date, guid, url, title are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

The following columns in the training set don't have a corresponding argument in `NNCFNetwork.forward` and have been ignored: date, guid, url, title. If date, guid, url, title are not expected by `NNCFNetwork.forward`,  you can safely ignore this message.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [ ]:
trainer.evaluate()

In [ ]:
test = dataset['validation']
test_labels = torch.tensor(test['label'])

In [ ]:
import time

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def sample(n):
  data = {
      'title':test['title'],
      'label':test['label']
  }
  return pd.DataFrame(data).sample(n)

def eval(model, d):
  
  total_preds = []
  total_labels = d['label']

  eval_start_time = time.time()
  for step, txt in tqdm(enumerate(d['title'])):
    output = torch.tensor([tokenizer.encode(txt)]).to(device)
    with torch.no_grad():
      preds = model(output).logits.cpu()

    pred = np.argmax(preds, axis=1).item()
    total_preds.append(pred)

  eval_end_time = time.time()
  eval_duration_time = eval_end_time - eval_start_time

  f1 = f1_score(total_labels, total_preds, average='macro')
  avg = accuracy_score(total_labels, total_preds) * 100
  
  print("Accuracy : {0:.4f}".format(avg))
  print("f1 score : {0:.4f}".format(f1))
  print("Evaluate total time (seconds) : {0:.1f}".format(eval_duration_time))

In [ ]:
from transformers import pipeline
from evaluate import evaluator

task_evaluator = evaluator("text-classification")


qa_pipe = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    max_seq_len=384,
    padding="max_length",
    truncation=True,
)
metric = task_evaluator.compute(model_or_pipeline=qa_pipe, data=dataset["validation"], metric="f1")

In [ ]:
d = sample(100) # test
eval(model, d)
eval(optimized_model, d)